# Testing of ipyleaflet

Alternative for maping is Folium

Installed for:
- python>=3.6
- ipyleaflet was installed via pip as conda broke with numpy exceptions
- all other install steps were followed via the install instructions

In [5]:
import warnings
warnings.filterwarnings(action='ignore', message="numpy.dtype size changed,")

import ipyleaflet
import datetime

In [22]:
from ipyleaflet import Map, basemaps, basemap_to_tiles

m = Map(
    layers=(basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, 
                             (datetime.datetime.today()-datetime.timedelta(1)).strftime('%Y-%m-%d')), ),
    center=(47, 238),
    zoom=5
)

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [10]:
(datetime.datetime.today()-datetime.timedelta(1)).strftime('%Y-%m-%d')

'2018-08-19'